In [1]:
from sqlalchemy import create_engine
import requests
import time
import pandas as pd
import datetime
import traceback

In [2]:
#reading in the file with the Apikey and login information. 
f = open('Open_weather_API_login.txt')
line = f.readlines() 
converted_list=[]

for element in line:
    converted_list.append(element.strip())


STATIONS_URI=converted_list[0]
APIKEY=converted_list[1]
host=converted_list[2]
db_name=converted_list[3]
user_name=converted_list[4]
password=converted_list[5]

In [3]:
#API parameters
lat = "53.3498"
lon = "-6.2603"
exclude="current,minutely"

#API URL
url="https://api.openweathermap.org/data/2.5/onecall?lat=%s&lon=%s&exclude=%s&appid=%s&units=metric" % (lat,lon,exclude,APIKEY)

#DataBase engine
engine = create_engine('mysql://'+ user_name+':'+ password+'@'+ host + '/'+ db_name)

In [4]:
def download_data(url):
    """Gets weather forecast for Dublin for 1 hour intervals over next 48 hours.
    Takes only weather information from return object and return information as Python array."""
    response = requests.get(url)
    data = response.json()

    if not response.ok:
        print("Error Code:", response.status_code)
        print("Reason:", response.reason)

    return data



def convert_data(data):
    """Takes only necessary data points from weather information, flattens internal dictionary objects, labels columns.
    Returns cleaned data as dataframe."""
    records = []
    for datum in data['hourly']:
        records.append({'date': datetime.datetime.fromtimestamp(datum['dt']),
        'temp': datum['temp'],
        'feels_like': datum['feels_like'],
        'wind_speed': datum['wind_speed'],
        'clouds_all': datum['clouds'],
        'weather_id' : datum['weather'][0]['id'],
        'description': datum['weather'][0]['description'],
        'main_description': datum['weather'][0]['main'],
        'icon': datum['weather'][0]['icon'],
        'sunrise': datetime.datetime.fromtimestamp(data['daily'][0]['sunrise']),
        'sunset': datetime.datetime.fromtimestamp(data['daily'][0]['sunset'])})
        #'sender_name':(data['alerts']['sender_name']),
         #   'event':(data['alerts'][0]['event']),
          #  'start': datetime.datetime.fromtimestamp(data['alerts'][0]['start']),
           # 'end': datetime.datetime.fromtimestamp(data['alerts'][0]['end']),  
            #'description': (data['alerts'][0]['description'])})
        
    return pd.DataFrame.from_records(records)


def main():
    #Downloads and converts data before sending to remote database.Repeats every three hours and overwrites table each time, so forecast data is always up to date."""
    try:
        data = download_data(url)
        df = convert_data(data)
        df.to_sql('weather_4_day_hourly_forecast', engine, if_exists='replace', index=False)
    except:
        print(traceback.format_exc())


if __name__ == "__main__":
    main()